In [1]:
import pandas as pd
import os,sys,time
import matplotlib.pyplot as plt
import seaborn as sns
import dask
import dask.dataframe as dd
dask.config.set({"temporary_directory": "E:\\"})
pd.options.display.max_columns = 9999
pd.options.display.max_rows = 9999
pd.options.display.width = 9999

sns.set()
'''
injurydf = dd.read_csv(r'InjuryRecord.csv')#, low_memory=False)
playerdf = dd.read_csv(r'PlayerTrackData.csv')#, low_memory=False)
playdf = dd.read_csv(r'PlayList.csv')#, low_memory=False)
'''
injurydf = pd.read_csv(r'InjuryRecord.csv', low_memory=False)
playerdf = pd.read_csv(r'PlayerTrackData.csv', low_memory=False)
playdf = pd.read_csv(r'PlayList.csv', low_memory=False)
print(injurydf.shape, playerdf.shape, playdf.shape)

(105, 9) (76366748, 9) (267005, 14)


dropna first dont care less data
memory not enuff

In [2]:
print(injurydf.shape, playerdf.shape, playdf.shape)
injurydf = injurydf.dropna()
playerdf = playerdf.dropna()
playdf = playdf.dropna()
print(injurydf.shape, playerdf.shape, playdf.shape)

(105, 9) (76366748, 9) (267005, 14)
(77, 9) (1839871, 9) (231996, 14)


investigate the relationship between the playing surface and the injury and performance of NFL athletes

In [3]:
injurydf.head()

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42
0,39873,39873-4,39873-4-32,Knee,Synthetic,1,1,1,1
1,46074,46074-7,46074-7-26,Knee,Natural,1,1,0,0
2,36557,36557-1,36557-1-70,Ankle,Synthetic,1,1,1,1
3,46646,46646-3,46646-3-30,Ankle,Natural,1,0,0,0
4,43532,43532-5,43532-5-69,Ankle,Synthetic,1,1,1,1


In [4]:
playerdf.head()

,PlayKey,time,event,x,y,dir,dis,o,s
0,26624-1-1,0.0,huddle_start_offense,87.46,28.93,288.24,0.01,262.33,0.13
41,26624-1-1,4.1,huddle_break_offense,87.48,28.92,325.16,0.00,266.96,0.17
85,26624-1-1,8.5,line_set,84.59,29.85,283.47,0.01,173.11,0.22
162,26624-1-1,16.2,ball_snap,84.39,29.74,94.89,0.01,191.09,1.46
196,26624-1-1,19.6,pass_forward,90.03,28.61,125.06,0.04,156.00,0.49


In [5]:
playdf.head()

,PlayerKey,GameID,PlayKey,RosterPosition,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Position,PositionGroup
0,26624,26624-1,26624-1-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,1,QB,QB
1,26624,26624-1,26624-1-2,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,2,QB,QB
2,26624,26624-1,26624-1-3,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Rush,3,QB,QB
3,26624,26624-1,26624-1-4,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Rush,4,QB,QB
4,26624,26624-1,26624-1-5,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,5,QB,QB


In [6]:
def summary(df):
    summarydf = pd.DataFrame(columns=df.columns)
    summarydf.loc["dtype"]= [df[col].dtype for col in df.columns.tolist()]
    summarydf.loc["nunique"]= [df[col].nunique() for col in df.columns.tolist()]
    summarydf.loc["unique"]= [df[col].unique() for col in df.columns.tolist()]
    summarydf.sort_values(by=['nunique'], axis=1)
    return summarydf


fix values

In [7]:
summary(injurydf).T

,dtype,nunique,unique
PlayerKey,int64,74,"[39873, 46074, 36557, 46646, 43532, 41145, 460..."
GameID,object,76,"[39873-4, 46074-7, 36557-1, 46646-3, 43532-5, ..."
PlayKey,object,76,"[39873-4-32, 46074-7-26, 36557-1-70, 46646-3-3..."
BodyPart,object,3,"[Knee, Ankle, Foot]"
Surface,object,2,"[Synthetic, Natural]"
DM_M1,int64,1,[1]
DM_M7,int64,2,"[1, 0]"
DM_M28,int64,2,"[1, 0]"
DM_M42,int64,2,"[1, 0]"


In [8]:
summary(playerdf).T

,dtype,nunique,unique
PlayKey,object,266960,"[26624-1-1, 26624-1-2, 26624-1-3, 26624-1-4, 2..."
time,float64,1299,"[0.0, 4.1, 8.5, 16.2, 19.6, 21.7, 22.4, 3.5, 8..."
event,object,80,"[huddle_start_offense, huddle_break_offense, l..."
x,float64,12077,"[87.46, 87.48, 84.59, 84.39, 90.03, 91.49, 91...."
y,float64,6543,"[28.93, 28.92, 29.85, 29.74, 28.61, 28.27, 27...."
dir,float64,36001,"[288.24, 325.16, 283.47, 94.89, 125.06, 161.33..."
dis,float64,174,"[0.01, 0.0, 0.04, 0.06, 0.1, 0.02, 0.03, 0.14,..."
o,float64,36001,"[262.33, 266.96, 173.11, 191.09, 156.0, 136.26..."
s,float64,1062,"[0.13, 0.17, 0.22, 1.46, 0.49, 0.57, 0.82, 0.0..."


In [9]:
summary(playdf).T

,dtype,nunique,unique
PlayerKey,int64,250,"[26624, 27363, 30068, 30953, 31070, 31266, 319..."
GameID,object,4971,"[26624-1, 26624-2, 26624-3, 26624-4, 26624-5, ..."
PlayKey,object,231996,"[26624-1-1, 26624-1-2, 26624-1-3, 26624-1-4, 2..."
RosterPosition,object,10,"[Quarterback, Wide Receiver, Linebacker, Runni..."
PlayerDay,int64,210,"[1, 11, 18, 25, 29, 39, 46, 53, 67, 74, 81, 88..."
PlayerGame,int64,32,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
StadiumType,object,26,"[Outdoor, Indoors, Oudoor, Outdoors, Open, Clo..."
FieldType,object,2,"[Synthetic, Natural]"
Temperature,int64,78,"[63, 85, 84, 61, 76, 73, 56, 62, 48, 66, 47, 5..."
Weather,object,63,"[Clear and warm, Mostly Cloudy, Sunny, Clear, ..."


In [10]:
playerdf['event'].unique()

array(['huddle_start_offense', 'huddle_break_offense', 'line_set',
       'ball_snap', 'pass_forward', 'pass_arrived',
       'pass_outcome_incomplete', 'pass_outcome_caught', 'first_contact',
       'out_of_bounds', 'man_in_motion', 'handoff', 'tackle',
       'penalty_flag', 'penalty_accepted', 'touchdown', 'shift',
       'qb_kneel', 'fumble', 'fumble_offense_recovered', 'lateral',
       'penalty_declined', 'qb_sack', 'pass_shovel',
       'pass_outcome_touchdown', 'run', 'pass_outcome_interception',
       'qb_strip_sack', 'two_point_conversion', 'pass_tipped',
       'fumble_defense_recovered', 'two_minute_warning', 'two_point_play',
       'snap_direct', 'play_action', 'qb_spike', 'pass_lateral',
       'touchback', 'timeout_tv', 'timeout', 'kickoff_play',
       'onside_kick', 'kick_received', 'safety', 'field_goal_attempt',
       'field_goal', 'punt_play', 'punt', 'punt_land', 'fair_catch',
       'punt_downed', 'punt_received', 'punt_fake', 'kickoff',
       'kickoff_land', 

In [11]:
playdf['RosterPosition'].unique()

array(['Quarterback', 'Wide Receiver', 'Linebacker', 'Running Back',
       'Defensive Lineman', 'Tight End', 'Safety', 'Cornerback',
       'Offensive Lineman', 'Kicker'], dtype=object)

In [12]:
playdf['StadiumType'].unique()

array(['Outdoor', 'Indoors', 'Oudoor', 'Outdoors', 'Open', 'Closed Dome',
       'Dome', 'Indoor', 'Domed', 'Retr. Roof-Closed', 'Retractable Roof',
       'Ourdoor', 'Indoor, Roof Closed', 'Retr. Roof - Closed', 'Bowl',
       'Outddors', 'Retr. Roof-Open', 'Indoor, Open Roof', 'Domed, Open',
       'Domed, open', 'Heinz Field', 'Cloudy', 'Retr. Roof - Open',
       'Retr. Roof Closed', 'Outdor', 'Outside'], dtype=object)

In [13]:
playdf['PlayType'].unique()

array(['Pass', 'Rush', '0', 'Kickoff Not Returned', 'Field Goal', 'Punt',
       'Punt Not Returned', 'Punt Returned', 'Kickoff Returned',
       'Kickoff', 'Extra Point'], dtype=object)

In [14]:
playdf['Weather'].unique()

array(['Clear and warm', 'Mostly Cloudy', 'Sunny', 'Clear', 'Cloudy',
       'Cloudy, fog started developing in 2nd quarter', 'Rain',
       'Partly Cloudy', 'Mostly cloudy', 'Cloudy and cold',
       'Cloudy and Cool', 'Rain Chance 40%', 'Controlled Climate',
       'Sunny and warm', 'Partly cloudy', 'Clear and Cool',
       'Clear and cold', 'Sunny and cold', 'Indoor', 'Partly Sunny',
       'N/A (Indoors)', 'Mostly Sunny', 'Indoors', 'Clear Skies',
       'Partly sunny', 'Showers', 'N/A Indoor', 'Sunny and clear', 'Snow',
       'Scattered Showers', 'Party Cloudy', 'Clear skies',
       'Rain likely, temps in low 40s.', 'Hazy', 'Partly Clouidy',
       'Sunny Skies', 'Overcast', 'Cloudy, 50% change of rain',
       'Partly clear', 'Mostly Coudy', 'Light Rain', '10% Chance of Rain',
       'Cloudy, chance of rain', 'Heat Index 95',
       'Sunny, highs to upper 80s', 'Sun & clouds',
       'Heavy lake effect snow', 'Fair', 'Mostly sunny', 'Cloudy, Rain',
       'Sunny, Windy', 'Mostl

drop na based weird description

In [15]:
playdf = playdf[playdf['PlayType'] != '0']
playdf = playdf[playdf['Position'] != 'Missing Data']
playdf = playdf[playdf['PositionGroup'] != 'Missing Data']
print(injurydf.shape, playerdf.shape, playdf.shape)

(77, 9) (1839871, 9) (231702, 14)


In [16]:
#bias assumption
'''
stadiumtype = {
    'Outdoor' : ['Oudoor', 'Outdoors', 'Outdoor', 'Outdor', 'Outside'],
    'Indoor' : ['Indoors', 'Indoor'],
    'Dome' : ['Dome', 'Domed', 'Domed, Open', 'Domed, open', 'Domed, Open'],
    'Roof Close' : ['Retr. Roof-Closed', 'Retractable Roof', 'Indoor, Roof Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed'],
    'Roof Open' : ['Retr. Roof-Open', 'Indoor, Open Roof', 'Retr. Roof - Open', 'Open'],
    'Heinz Field' : ['Heinz Field'],
    'Cloudy' : ['Cloudy'],
    
    
    'Outdoor', 'Indoors', 'Oudoor', 'Outdoors', 'Open', 'Closed Dome',
       'Dome', 'Indoor', 'Domed', 'Retr. Roof-Closed', 'Retractable Roof',
       'Ourdoor', 'Indoor, Roof Closed', 'Retr. Roof - Closed', 'Bowl',
       'Outddors', 'Retr. Roof-Open', 'Indoor, Open Roof', 'Domed, Open',
       'Domed, open', 'Heinz Field', 'Cloudy', 'Retr. Roof - Open',
       'Retr. Roof Closed', 'Outdor', 'Outside'
}
def fixstadiumtype(row):
'''

"\nstadiumtype = {\n    'Outdoor' : ['Oudoor', 'Outdoors', 'Outdoor', 'Outdor', 'Outside'],\n    'Indoor' : ['Indoors', 'Indoor'],\n    'Dome' : ['Dome', 'Domed', 'Domed, Open', 'Domed, open', 'Domed, Open'],\n    'Roof Close' : ['Retr. Roof-Closed', 'Retractable Roof', 'Indoor, Roof Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed'],\n    'Roof Open' : ['Retr. Roof-Open', 'Indoor, Open Roof', 'Retr. Roof - Open', 'Open'],\n    'Heinz Field' : ['Heinz Field'],\n    'Cloudy' : ['Cloudy'],\n    \n    \n    'Outdoor', 'Indoors', 'Oudoor', 'Outdoors', 'Open', 'Closed Dome',\n       'Dome', 'Indoor', 'Domed', 'Retr. Roof-Closed', 'Retractable Roof',\n       'Ourdoor', 'Indoor, Roof Closed', 'Retr. Roof - Closed', 'Bowl',\n       'Outddors', 'Retr. Roof-Open', 'Indoor, Open Roof', 'Domed, Open',\n       'Domed, open', 'Heinz Field', 'Cloudy', 'Retr. Roof - Open',\n       'Retr. Roof Closed', 'Outdor', 'Outside'\n}\ndef fixstadiumtype(row):\n"

In [17]:
#remove fuzzy features
playdf = playdf.drop(['Weather','StadiumType'],axis=1)

In [27]:
test = pd.merge(playdf,injurydf, on=["PlayKey","GameID","PlayerKey"], how="left")
test.shape

(231703, 18)

In [26]:
pd.isna(test).sum()

PlayerKey         0
GameID            0
PlayKey           0
RosterPosition    7
PlayerDay         7
PlayerGame        7
FieldType         7
Temperature       7
PlayType          7
PlayerGamePlay    7
Position          7
PositionGroup     7
BodyPart          0
Surface           0
DM_M1             0
DM_M7             0
DM_M28            0
DM_M42            0
dtype: int64

In [20]:
test = pd.merge(test,playerdf, on="PlayKey", how="right")
test.shape

(1839877, 26)

In [21]:
pd.isna(test).sum()

PlayerKey          243157
GameID             243157
PlayKey                 0
RosterPosition     243157
PlayerDay          243157
PlayerGame         243157
FieldType          243157
Temperature        243157
PlayType           243157
PlayerGamePlay     243157
Position           243157
PositionGroup      243157
BodyPart          1839429
Surface           1839429
DM_M1             1839429
DM_M7             1839429
DM_M28            1839429
DM_M42            1839429
time                    0
event                   0
x                       0
y                       0
dir                     0
dis                     0
o                       0
s                       0
dtype: int64

In [22]:
test.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448 entries, 46932 to 1592128
Data columns (total 26 columns):
PlayerKey         448 non-null float64
GameID            448 non-null object
PlayKey           448 non-null object
RosterPosition    448 non-null object
PlayerDay         448 non-null float64
PlayerGame        448 non-null float64
FieldType         448 non-null object
Temperature       448 non-null float64
PlayType          448 non-null object
PlayerGamePlay    448 non-null float64
Position          448 non-null object
PositionGroup     448 non-null object
BodyPart          448 non-null object
Surface           448 non-null object
DM_M1             448 non-null float64
DM_M7             448 non-null float64
DM_M28            448 non-null float64
DM_M42            448 non-null float64
time              448 non-null float64
event             448 non-null object
x                 448 non-null float64
y                 448 non-null float64
dir               448 non-null float64


In [23]:
test.dropna().to_csv('clean_data.csv',index=False)

In [24]:
#sns.heatmap(test.corr())

In [ ]:
cleandf = test.dropna()